In [ ]:
#Scenario
#1. test normalisasi suatu data, apakah distribusinya normal atau tidak-> menggunakn Shapiro-Wilk Test
#2. Kalau normal bisa menggunakan t-test, kalau tidak menggunakn Wilcoxon Effect Size 
#3. Nah keduanya harus ada sample minim 3-5x percobaan untuk membuktikan bahwa nilai akurasinya itu stabil
#4. Inisiatif nya bisa dibuat k-fold cross validation dengan membagi data  menjadi 5 bagian dengan iterasi terakhir sebagai testing
#5. Hasilnya akan ada 5 nilai akurasi di setiap modelnya yang mana akan dibandingkan antar model satu dengan model lain


Data: Using Undersampling Dataset

In [2]:
#import package and library
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

In [21]:
from sklearn.model_selection import StratifiedKFold
datasdp = pd.read_csv('D:\AAAAKK SKRIPSWEETT\Bismillah Code Skripsi V2\Dataset\dataset_sdp_undersampling.csv', delimiter=";")

X = datasdp.drop(columns=['bugs'])
y = datasdp['bugs']

X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_base, X_train_meta, y_train_base, y_train_meta = train_test_split(X_train, y_train, test_size=0.5, random_state=42)

counts_training = y_train.value_counts()
print("data distribiution data training: ", counts_training)
counts_testing = y_train_base.value_counts()
print("data distribiution data testing: ", counts_testing)

#insialisasi k-fold validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_base, y_train_base), 1):
    X_train_fold, X_val_fold = X_train_base.iloc[train_idx], X_train_base.iloc[val_idx]
    y_train_fold, y_val_fold = y_train_base.iloc[train_idx], y_train_base.iloc[val_idx]

    print(f"Fold {fold} - Data Training Undersampled: {y_train.value_counts().to_dict()}")
    print(f"Fold {fold} - Data Validation: {y_val_fold.value_counts().to_dict()}")

data distribiution data training:  bugs
0    593
1    458
Name: count, dtype: int64
data distribiution data testing:  bugs
0    286
1    239
Name: count, dtype: int64
Fold 1 - Data Training Undersampled: {0: 593, 1: 458}
Fold 1 - Data Validation: {0: 58, 1: 47}
Fold 2 - Data Training Undersampled: {0: 593, 1: 458}
Fold 2 - Data Validation: {0: 57, 1: 48}
Fold 3 - Data Training Undersampled: {0: 593, 1: 458}
Fold 3 - Data Validation: {0: 57, 1: 48}
Fold 4 - Data Training Undersampled: {0: 593, 1: 458}
Fold 4 - Data Validation: {0: 57, 1: 48}
Fold 5 - Data Training Undersampled: {0: 593, 1: 458}
Fold 5 - Data Validation: {0: 57, 1: 48}


In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, f1_score, precision_score
from scipy.stats import shapiro, ttest_rel, wilcoxon
import numpy as np

# Inisialisasi model
randomforest_classifier = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_leaf=2,
    criterion='gini'
)

# List untuk menyimpan metrik setiap fold
roc_auc_scores = []
accuracy_scores = []
recall_scores = []
f1_scores = []
precision_scores = []

# Loop untuk K-Fold
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_base, y_train_base), 1):
    print(f"Training fold {fold}...")

    # Pisahkan data training dan validasi pada fold ini
    X_train_fold, X_val_fold = X_train_base.iloc[train_idx], X_train_base.iloc[val_idx]
    y_train_fold, y_val_fold = y_train_base.iloc[train_idx], y_train_base.iloc[val_idx]

    # Train model
    randomforest_classifier.fit(X_train_fold, y_train_fold)

    # Prediksi pada data validasi
    val_predictions = randomforest_classifier.predict(X_val_fold)

    # Hitung metrik evaluasi    
    recall = recall_score(y_val_fold, val_predictions)
    f1score = f1_score(y_val_fold, val_predictions)
    precision = precision_score(y_val_fold, val_predictions)
    roc_auc = roc_auc_score(y_val_fold, val_predictions)
    accuracy = accuracy_score(y_val_fold, val_predictions)

    # Simpan hasil metrik di list
    recall_scores.append(recall)
    f1_scores.append(f1score)
    precision_scores.append(precision)
    roc_auc_scores.append(roc_auc)
    accuracy_scores.append(accuracy)

    # Print hasil per fold
    print(f'Fold {fold} - Recall: {recall:.4f}')
    print(f'Fold {fold} - F1 Score: {f1score:.4f}')
    print(f'Fold {fold} - Precision: {precision:.4f}')
    print(f'Fold {fold} - ROC AUC Score: {roc_auc:.4f}')
    print(f'Fold {fold} - Accuracy: {accuracy:.4f}')



Training fold 1...
Fold 1 - Recall: 0.8511
Fold 1 - F1 Score: 0.8696
Fold 1 - Precision: 0.8889
Fold 1 - ROC AUC Score: 0.8824
Fold 1 - Accuracy: 0.8857
Training fold 2...
Fold 2 - Recall: 0.7708
Fold 2 - F1 Score: 0.8043
Fold 2 - Precision: 0.8409
Fold 2 - ROC AUC Score: 0.8240
Fold 2 - Accuracy: 0.8286
Training fold 3...
Fold 3 - Recall: 0.7708
Fold 3 - F1 Score: 0.8222
Fold 3 - Precision: 0.8810
Fold 3 - ROC AUC Score: 0.8416
Fold 3 - Accuracy: 0.8476
Training fold 4...
Fold 4 - Recall: 0.7708
Fold 4 - F1 Score: 0.7872
Fold 4 - Precision: 0.8043
Fold 4 - ROC AUC Score: 0.8065
Fold 4 - Accuracy: 0.8095
Training fold 5...
Fold 5 - Recall: 0.8750
Fold 5 - F1 Score: 0.9032
Fold 5 - Precision: 0.9333
Fold 5 - ROC AUC Score: 0.9112
Fold 5 - Accuracy: 0.9143


In [24]:
#2. extra trees
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, f1_score, precision_score

et_model = ExtraTreesClassifier(
    n_estimators=100,
    max_depth=10,
    min_samples_leaf=2,
    criterion='gini'
)

# List untuk menyimpan metrik setiap fold
roc_auc_scores = []
accuracy_scores = []
recall_scores = []
f1_scores = []
precision_scores = []

# Loop untuk K-Fold
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_base, y_train_base), 1):
    print(f"Training fold {fold}...")

    # Pisahkan data training dan validasi pada fold ini
    X_train_fold, X_val_fold = X_train_base.iloc[train_idx], X_train_base.iloc[val_idx]
    y_train_fold, y_val_fold = y_train_base.iloc[train_idx], y_train_base.iloc[val_idx]

    # Train model
    et_model.fit(X_train_fold, y_train_fold)

    # Prediksi pada data validasi
    val_predictions = et_model.predict(X_val_fold)

    # Hitung metrik evaluasi
    recall = recall_score(y_val_fold, val_predictions)
    f1score = f1_score(y_val_fold, val_predictions)
    precision = precision_score(y_val_fold, val_predictions)
    roc_auc = roc_auc_score(y_val_fold, val_predictions)
    accuracy = accuracy_score(y_val_fold, val_predictions)

    # Simpan hasil metrik di list
    recall_scores.append(recall)
    f1_scores.append(f1score)
    precision_scores.append(precision)
    roc_auc_scores.append(roc_auc)
    accuracy_scores.append(accuracy)

    # Print hasil per fold
    print(f'Fold {fold} - Recall: {recall:.4f}')
    print(f'Fold {fold} - F1 Score: {f1score:.4f}')
    print(f'Fold {fold} - Precision: {precision:.4f}')
    print(f'Fold {fold} - ROC AUC Score: {roc_auc:.4f}')
    print(f'Fold {fold} - Accuracy: {accuracy:.4f}')



Training fold 1...
Fold 1 - Recall: 0.7660
Fold 1 - F1 Score: 0.8090
Fold 1 - Precision: 0.8571
Fold 1 - ROC AUC Score: 0.8313
Fold 1 - Accuracy: 0.8381
Training fold 2...
Fold 2 - Recall: 0.6667
Fold 2 - F1 Score: 0.7111
Fold 2 - Precision: 0.7619
Fold 2 - ROC AUC Score: 0.7456
Fold 2 - Accuracy: 0.7524
Training fold 3...
Fold 3 - Recall: 0.7708
Fold 3 - F1 Score: 0.8315
Fold 3 - Precision: 0.9024
Fold 3 - ROC AUC Score: 0.8503
Fold 3 - Accuracy: 0.8571
Training fold 4...
Fold 4 - Recall: 0.7083
Fold 4 - F1 Score: 0.7473
Fold 4 - Precision: 0.7907
Fold 4 - ROC AUC Score: 0.7752
Fold 4 - Accuracy: 0.7810
Training fold 5...
Fold 5 - Recall: 0.8125
Fold 5 - F1 Score: 0.8571
Fold 5 - Precision: 0.9070
Fold 5 - ROC AUC Score: 0.8712
Fold 5 - Accuracy: 0.8762


In [25]:
#3. Adapative Boosting (AdaaBoost) using smote technique

from sklearn.ensemble  import  AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score, accuracy_score

adaboost_model = AdaBoostClassifier(
    estimator=DecisionTreeClassifier(max_depth=1), 
    n_estimators=5000,
    random_state=1
)

# List untuk menyimpan metrik setiap fold
roc_auc_scores = []
accuracy_scores = []
recall_scores = []
f1_scores = []
precision_scores = []

# Loop untuk K-Fold
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_base, y_train_base), 1):
    print(f"Training fold {fold}...")

    # Pisahkan data training dan validasi pada fold ini
    X_train_fold, X_val_fold = X_train_base.iloc[train_idx], X_train_base.iloc[val_idx]
    y_train_fold, y_val_fold = y_train_base.iloc[train_idx], y_train_base.iloc[val_idx]

    # Train model
    adaboost_model.fit(X_train_fold, y_train_fold)

    # Prediksi pada data validasi
    val_predictions = adaboost_model.predict(X_val_fold)

    # Hitung metrik evaluasi
    recall = recall_score(y_val_fold, val_predictions)
    f1score = f1_score(y_val_fold, val_predictions)
    precision = precision_score(y_val_fold, val_predictions)
    roc_auc = roc_auc_score(y_val_fold, val_predictions)
    accuracy = accuracy_score(y_val_fold, val_predictions)

    # Simpan hasil metrik di list
    recall_scores.append(recall)
    f1_scores.append(f1score)
    precision_scores.append(precision)
    roc_auc_scores.append(roc_auc)
    accuracy_scores.append(accuracy)

    # Print hasil per fold
    print(f'Fold {fold} - Recall: {recall:.4f}')
    print(f'Fold {fold} - F1 Score: {f1score:.4f}')
    print(f'Fold {fold} - Precision: {precision:.4f}')
    print(f'Fold {fold} - ROC AUC Score: {roc_auc:.4f}')
    print(f'Fold {fold} - Accuracy: {accuracy:.4f}')



Training fold 1...


c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Fold 1 - Recall: 0.8085
Fold 1 - F1 Score: 0.7835
Fold 1 - Precision: 0.7600
Fold 1 - ROC AUC Score: 0.8008
Fold 1 - Accuracy: 0.8000
Training fold 2...


c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Fold 2 - Recall: 0.7500
Fold 2 - F1 Score: 0.7579
Fold 2 - Precision: 0.7660
Fold 2 - ROC AUC Score: 0.7785
Fold 2 - Accuracy: 0.7810
Training fold 3...


...

Fold 3 - Recall: 0.7708
Fold 3 - F1 Score: 0.7400
Fold 3 - Precision: 0.7115
Fold 3 - ROC AUC Score: 0.7538
Fold 3 - Accuracy: 0.7524
Training fold 4...


c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Fold 4 - Recall: 0.7917
Fold 4 - F1 Score: 0.8085
Fold 4 - Precision: 0.8261
Fold 4 - ROC AUC Score: 0.8257
Fold 4 - Accuracy: 0.8286
Training fold 5...


c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Fold 5 - Recall: 0.7917
Fold 5 - F1 Score: 0.7835
Fold 5 - Precision: 0.7755
Fold 5 - ROC AUC Score: 0.7993
Fold 5 - Accuracy: 0.8000


In [26]:
#4. Gradient Boosting 

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, f1_score, precision_score, classification_report

gb_model = GradientBoostingClassifier(
    n_estimators=100, 
    learning_rate=0.1,
    min_samples_leaf=2,
    max_depth=5,
    loss='exponential'
)

# List untuk menyimpan metrik setiap fold
roc_auc_scores = []
accuracy_scores = []
recall_scores = []
f1_scores = []
precision_scores = []

# Loop untuk K-Fold
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_base, y_train_base), 1):
    print(f"Training fold {fold}...")

    # Pisahkan data training dan validasi pada fold ini
    X_train_fold, X_val_fold = X_train_base.iloc[train_idx], X_train_base.iloc[val_idx]
    y_train_fold, y_val_fold = y_train_base.iloc[train_idx], y_train_base.iloc[val_idx]

    # Train model
    gb_model.fit(X_train_fold, y_train_fold)

    # Prediksi pada data validasi
    val_predictions = gb_model.predict(X_val_fold)

    # Hitung metrik evaluasi
    recall = recall_score(y_val_fold, val_predictions)
    f1score = f1_score(y_val_fold, val_predictions)
    precision = precision_score(y_val_fold, val_predictions)
    roc_auc = roc_auc_score(y_val_fold, val_predictions)
    accuracy = accuracy_score(y_val_fold, val_predictions)

    # Simpan hasil metrik di list
    recall_scores.append(recall)
    f1_scores.append(f1score)
    precision_scores.append(precision)
    roc_auc_scores.append(roc_auc)
    accuracy_scores.append(accuracy)

    # Print hasil per fold
    print(f'Fold {fold} - Recall: {recall:.4f}')
    print(f'Fold {fold} - F1 Score: {f1score:.4f}')
    print(f'Fold {fold} - Precision: {precision:.4f}')
    print(f'Fold {fold} - ROC AUC Score: {roc_auc:.4f}')
    print(f'Fold {fold} - Accuracy: {accuracy:.4f}')


Training fold 1...
Fold 1 - Recall: 0.8511
Fold 1 - F1 Score: 0.8791
Fold 1 - Precision: 0.9091
Fold 1 - ROC AUC Score: 0.8910
Fold 1 - Accuracy: 0.8952
Training fold 2...
Fold 2 - Recall: 0.7917
Fold 2 - F1 Score: 0.8085
Fold 2 - Precision: 0.8261
Fold 2 - ROC AUC Score: 0.8257
Fold 2 - Accuracy: 0.8286
Training fold 3...
Fold 3 - Recall: 0.7292
Fold 3 - F1 Score: 0.7865
Fold 3 - Precision: 0.8537
Fold 3 - ROC AUC Score: 0.8120
Fold 3 - Accuracy: 0.8190
Training fold 4...
Fold 4 - Recall: 0.7917
Fold 4 - F1 Score: 0.8261
Fold 4 - Precision: 0.8636
Fold 4 - ROC AUC Score: 0.8432
Fold 4 - Accuracy: 0.8476
Training fold 5...
Fold 5 - Recall: 0.8542
Fold 5 - F1 Score: 0.8632
Fold 5 - Precision: 0.8723
Fold 5 - ROC AUC Score: 0.8745
Fold 5 - Accuracy: 0.8762


In [28]:
#5. Histogram Gradient Boosting

from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, f1_score, precision_score, classification_report

hgb_model = HistGradientBoostingClassifier(
    max_iter=100,
    learning_rate=0.1,
    min_samples_leaf=2,
    max_depth=5
)

# List untuk menyimpan metrik setiap fold
roc_auc_scores = []
accuracy_scores = []
recall_scores = []
f1_scores = []
precision_scores = []

# Loop untuk K-Fold
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_base, y_train_base), 1):
    print(f"Training fold {fold}...")

    # Pisahkan data training dan validasi pada fold ini
    X_train_fold, X_val_fold = X_train_base.iloc[train_idx], X_train_base.iloc[val_idx]
    y_train_fold, y_val_fold = y_train_base.iloc[train_idx], y_train_base.iloc[val_idx]

    # Train model
    hgb_model.fit(X_train_fold, y_train_fold)

    # Prediksi pada data validasi
    val_predictions = hgb_model.predict(X_val_fold)

    # Hitung metrik evaluasi
    recall = recall_score(y_val_fold, val_predictions)
    f1score = f1_score(y_val_fold, val_predictions)
    precision = precision_score(y_val_fold, val_predictions)
    roc_auc = roc_auc_score(y_val_fold, val_predictions)
    accuracy = accuracy_score(y_val_fold, val_predictions)

    # Simpan hasil metrik di list
    recall_scores.append(recall)
    f1_scores.append(f1score)
    precision_scores.append(precision)
    roc_auc_scores.append(roc_auc)
    accuracy_scores.append(accuracy)

    # Print hasil per fold
    print(f'Fold {fold} - Recall: {recall:.4f}')
    print(f'Fold {fold} - F1 Score: {f1score:.4f}')
    print(f'Fold {fold} - Precision: {precision:.4f}')
    print(f'Fold {fold} - ROC AUC Score: {roc_auc:.4f}')
    print(f'Fold {fold} - Accuracy: {accuracy:.4f}')

Training fold 1...
Fold 1 - Recall: 0.8511
Fold 1 - F1 Score: 0.8696
Fold 1 - Precision: 0.8889
Fold 1 - ROC AUC Score: 0.8824
Fold 1 - Accuracy: 0.8857
Training fold 2...
Fold 2 - Recall: 0.8125
Fold 2 - F1 Score: 0.8041
Fold 2 - Precision: 0.7959
Fold 2 - ROC AUC Score: 0.8185
Fold 2 - Accuracy: 0.8190
Training fold 3...
Fold 3 - Recall: 0.7500
Fold 3 - F1 Score: 0.7742
Fold 3 - Precision: 0.8000
Fold 3 - ROC AUC Score: 0.7961
Fold 3 - Accuracy: 0.8000
Training fold 4...
Fold 4 - Recall: 0.7917
Fold 4 - F1 Score: 0.8172
Fold 4 - Precision: 0.8444
Fold 4 - ROC AUC Score: 0.8344
Fold 4 - Accuracy: 0.8381
Training fold 5...
Fold 5 - Recall: 0.8750
Fold 5 - F1 Score: 0.8842
Fold 5 - Precision: 0.8936
Fold 5 - ROC AUC Score: 0.8936
Fold 5 - Accuracy: 0.8952


In [29]:
#6. XGBoost 

from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, f1_score, precision_score, classification_report

xgb_model = XGBClassifier(
    n_estimators=100, 
    max_depth=5, 
    min_child_weight=1, 
    max_delta_step=0, 
    random_state=42)

# List untuk menyimpan metrik setiap fold
roc_auc_scores = []
accuracy_scores = []
recall_scores = []
f1_scores = []
precision_scores = []

# Loop untuk K-Fold
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_base, y_train_base), 1):
    print(f"Training fold {fold}...")

    # Pisahkan data training dan validasi pada fold ini
    X_train_fold, X_val_fold = X_train_base.iloc[train_idx], X_train_base.iloc[val_idx]
    y_train_fold, y_val_fold = y_train_base.iloc[train_idx], y_train_base.iloc[val_idx]

    # Train model
    xgb_model.fit(X_train_fold, y_train_fold)

    # Prediksi pada data validasi
    val_predictions = xgb_model.predict(X_val_fold)

    # Hitung metrik evaluasi
    recall = recall_score(y_val_fold, val_predictions)
    f1score = f1_score(y_val_fold, val_predictions)
    precision = precision_score(y_val_fold, val_predictions)
    roc_auc = roc_auc_score(y_val_fold, val_predictions)
    accuracy = accuracy_score(y_val_fold, val_predictions)

    # Simpan hasil metrik di list
    recall_scores.append(recall)
    f1_scores.append(f1score)
    precision_scores.append(precision)
    roc_auc_scores.append(roc_auc)
    accuracy_scores.append(accuracy)

    # Print hasil per fold
    print(f'Fold {fold} - Recall: {recall:.4f}')
    print(f'Fold {fold} - F1 Score: {f1score:.4f}')
    print(f'Fold {fold} - Precision: {precision:.4f}')
    print(f'Fold {fold} - ROC AUC Score: {roc_auc:.4f}')
    print(f'Fold {fold} - Accuracy: {accuracy:.4f}')

Training fold 1...
Fold 1 - Recall: 0.7660
Fold 1 - F1 Score: 0.8000
Fold 1 - Precision: 0.8372
Fold 1 - ROC AUC Score: 0.8226
Fold 1 - Accuracy: 0.8286
Training fold 2...
Fold 2 - Recall: 0.8125
Fold 2 - F1 Score: 0.7800
Fold 2 - Precision: 0.7500
Fold 2 - ROC AUC Score: 0.7922
Fold 2 - Accuracy: 0.7905
Training fold 3...
Fold 3 - Recall: 0.7292
Fold 3 - F1 Score: 0.7692
Fold 3 - Precision: 0.8140
Fold 3 - ROC AUC Score: 0.7944
Fold 3 - Accuracy: 0.8000
Training fold 4...
Fold 4 - Recall: 0.7708
Fold 4 - F1 Score: 0.8132
Fold 4 - Precision: 0.8605
Fold 4 - ROC AUC Score: 0.8328
Fold 4 - Accuracy: 0.8381
Training fold 5...
Fold 5 - Recall: 0.8542
Fold 5 - F1 Score: 0.8913
Fold 5 - Precision: 0.9318
Fold 5 - ROC AUC Score: 0.9008
Fold 5 - Accuracy: 0.9048


In [30]:
#7. Categorical Boosting (CatBoost)

from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, f1_score, precision_score, classification_report

cb_model = CatBoostClassifier(
    n_estimators=1000,
    loss_function='Logloss',
    learning_rate=0.1,
    depth=6,
    min_data_in_leaf=2,
    random_seed=1,
    logging_level='Silent')

# List untuk menyimpan metrik setiap fold
roc_auc_scores = []
accuracy_scores = []
recall_scores = []
f1_scores = []
precision_scores = []

# Loop untuk K-Fold
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_base, y_train_base), 1):
    print(f"Training fold {fold}...")

    # Pisahkan data training dan validasi pada fold ini
    X_train_fold, X_val_fold = X_train_base.iloc[train_idx], X_train_base.iloc[val_idx]
    y_train_fold, y_val_fold = y_train_base.iloc[train_idx], y_train_base.iloc[val_idx]

    # Train model
    cb_model.fit(X_train_fold, y_train_fold)

    # Prediksi pada data validasi
    val_predictions = cb_model.predict(X_val_fold)

    # Hitung metrik evaluasi
    recall = recall_score(y_val_fold, val_predictions)
    f1score = f1_score(y_val_fold, val_predictions)
    precision = precision_score(y_val_fold, val_predictions)
    roc_auc = roc_auc_score(y_val_fold, val_predictions)
    accuracy = accuracy_score(y_val_fold, val_predictions)

    # Simpan hasil metrik di list
    recall_scores.append(recall)
    f1_scores.append(f1score)
    precision_scores.append(precision)
    roc_auc_scores.append(roc_auc)
    accuracy_scores.append(accuracy)

    # Print hasil per fold
    print(f'Fold {fold} - Recall: {recall:.4f}')
    print(f'Fold {fold} - F1 Score: {f1score:.4f}')
    print(f'Fold {fold} - Precision: {precision:.4f}')
    print(f'Fold {fold} - ROC AUC Score: {roc_auc:.4f}')
    print(f'Fold {fold} - Accuracy: {accuracy:.4f}')

Training fold 1...
Fold 1 - Recall: 0.8723
Fold 1 - F1 Score: 0.8723
Fold 1 - Precision: 0.8723
Fold 1 - ROC AUC Score: 0.8844
Fold 1 - Accuracy: 0.8857
Training fold 2...
Fold 2 - Recall: 0.7708
Fold 2 - F1 Score: 0.7789
Fold 2 - Precision: 0.7872
Fold 2 - ROC AUC Score: 0.7977
Fold 2 - Accuracy: 0.8000
Training fold 3...
Fold 3 - Recall: 0.7917
Fold 3 - F1 Score: 0.8172
Fold 3 - Precision: 0.8444
Fold 3 - ROC AUC Score: 0.8344
Fold 3 - Accuracy: 0.8381
Training fold 4...
Fold 4 - Recall: 0.7917
Fold 4 - F1 Score: 0.8085
Fold 4 - Precision: 0.8261
Fold 4 - ROC AUC Score: 0.8257
Fold 4 - Accuracy: 0.8286
Training fold 5...
Fold 5 - Recall: 0.8750
Fold 5 - F1 Score: 0.9032
Fold 5 - Precision: 0.9333
Fold 5 - ROC AUC Score: 0.9112
Fold 5 - Accuracy: 0.9143


In [31]:
#create stacking ensemble
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

estimators_base_model = [
    ('randomforest', randomforest_classifier),
    ('extratrees', et_model),
    ('adaboost', adaboost_model),
    ('gradboost', gb_model),
    ('hgboost', hgb_model),
    ('xgboost', xgb_model),
    ('catboost', cb_model), 
]

#stacked model with  base model and logistic
stack_model = StackingClassifier(estimators=estimators_base_model, final_estimator=LogisticRegression(), passthrough=False)

# List untuk menyimpan metrik setiap fold
roc_auc_scores = []
accuracy_scores = []
recall_scores = []
f1_scores = []
precision_scores = []

# Loop untuk K-Fold
for fold, (train_idx, val_idx) in enumerate(skf.split(X_train_base, y_train_base), 1):
    print(f"Training fold {fold}...")

    # Pisahkan data training dan validasi pada fold ini
    X_train_fold, X_val_fold = X_train_base.iloc[train_idx], X_train_base.iloc[val_idx]
    y_train_fold, y_val_fold = y_train_base.iloc[train_idx], y_train_base.iloc[val_idx]

    # Train model
    stack_model.fit(X_train_fold, y_train_fold)

    # Prediksi pada data validasi
    val_predictions = stack_model.predict(X_val_fold)

    # Hitung metrik evaluasi
    recall = recall_score(y_val_fold, val_predictions)
    f1score = f1_score(y_val_fold, val_predictions)
    precision = precision_score(y_val_fold, val_predictions)
    roc_auc = roc_auc_score(y_val_fold, val_predictions)
    accuracy = accuracy_score(y_val_fold, val_predictions)

    # Simpan hasil metrik di list
    recall_scores.append(recall)
    f1_scores.append(f1score)
    precision_scores.append(precision)
    roc_auc_scores.append(roc_auc)
    accuracy_scores.append(accuracy)

    # Print hasil per fold
    print(f'Fold {fold} - Recall: {recall:.4f}')
    print(f'Fold {fold} - F1 Score: {f1score:.4f}')
    print(f'Fold {fold} - Precision: {precision:.4f}')
    print(f'Fold {fold} - ROC AUC Score: {roc_auc:.4f}')
    print(f'Fold {fold} - Accuracy: {accuracy:.4f}')

Training fold 1...


c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated a

Fold 1 - Recall: 0.8723
Fold 1 - F1 Score: 0.8913
Fold 1 - Precision: 0.9111
Fold 1 - ROC AUC Score: 0.9017
Fold 1 - Accuracy: 0.9048
Training fold 2...


c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated a

Fold 2 - Recall: 0.7500
Fold 2 - F1 Score: 0.7660
Fold 2 - Precision: 0.7826
Fold 2 - ROC AUC Score: 0.7873
Fold 2 - Accuracy: 0.7905
Training fold 3...


...

Fold 3 - Recall: 0.7917
Fold 3 - F1 Score: 0.8172
Fold 3 - Precision: 0.8444
Fold 3 - ROC AUC Score: 0.8344
Fold 3 - Accuracy: 0.8381
Training fold 4...


c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated a

Fold 4 - Recall: 0.7917
Fold 4 - F1 Score: 0.8172
Fold 4 - Precision: 0.8444
Fold 4 - ROC AUC Score: 0.8344
Fold 4 - Accuracy: 0.8381
Training fold 5...


c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\HP\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated a

Fold 5 - Recall: 0.8750
Fold 5 - F1 Score: 0.9032
Fold 5 - Precision: 0.9333
Fold 5 - ROC AUC Score: 0.9112
Fold 5 - Accuracy: 0.9143


Normality testing using Shapiro Wilk and T-test

In [34]:
import numpy as np
from scipy.stats import shapiro, ttest_rel, wilcoxon

# Data hasil K-Fold
model_RF = np.array([0.8824, 0.8240, 0.8416, 0.8065, 0.9112])
model_ET = np.array([0.8313, 0.7456, 0.8503, 0.7752, 0.8712])
model_ADA = np.array([0.8008, 0.7785, 0.7538, 0.8257, 0.7993])
model_GB = np.array([0.8910, 0.8257, 0.8120, 0.8432, 0.8745])
model_HGB = np.array([0.8824, 0.8185, 0.7961, 0.8344, 0.8396])
model_XGB = np.array([0.8226, 0.7922, 0.7944, 0.8328, 0.9008])
model_CB = np.array([0.8844, 0.7977, 0.8344, 0.8257, 0.9112])
model_STA = np.array([0.9017, 0.7873, 0.8344, 0.8381, 0.9112])  

# List model untuk perbandingan
models = {
    "RF": model_RF,
    "ET": model_ET,
    "ADA": model_ADA,
    "GB": model_GB,
    "HGB": model_HGB,
    "XGB": model_XGB,
    "CB": model_CB,
    "STA": model_STA
}

# Shapiro-Wilk Test untuk normalitas
print("\n=== Shapiro-Wilk Normality Test ===")
for name, scores in models.items():
    stat, p = shapiro(scores)
    print(f"{name}: W={stat:.4f}, p-value={p:.4f} ({'Normal' if p > 0.05 else 'Non-normal'})")

# Uji T-Test & Wilcoxon antara RF dan model lain
print("\n=== Paired T-Test & Wilcoxon Test Against RF ===")
for name, scores in models.items():
    if name == "STA":
        continue  # Skip diri sendiri
    
    # Uji normalitas kedua model
    _, p_rf = shapiro(model_STA)
    _, p_model = shapiro(scores)

    if p_rf > 0.05 and p_model > 0.05:
        # Jika normal, gunakan T-Test
        t_stat, p_ttest = ttest_rel(model_STA, scores)
        print(f"T-Test STA vs {name}: t-stat={t_stat:.4f}, p-value={p_ttest:.4f} ({'Significant' if p_ttest < 0.05 else 'Not Significant'})")
    else:
        # Jika tidak normal, gunakan Wilcoxon
        w_stat, p_wilcoxon = wilcoxon(model_RF, scores)
        print(f"Wilcoxon STA vs {name}: W={w_stat:.4f}, p-value={p_wilcoxon:.4f} ({'Significant' if p_wilcoxon < 0.05 else 'Not Significant'})")



=== Shapiro-Wilk Normality Test ===
RF: W=0.9483, p-value=0.7250 (Normal)
ET: W=0.9325, p-value=0.6139 (Normal)
ADA: W=0.9724, p-value=0.8906 (Normal)
GB: W=0.9480, p-value=0.7226 (Normal)
HGB: W=0.9629, p-value=0.8278 (Normal)
XGB: W=0.8548, p-value=0.2100 (Normal)
CB: W=0.9451, p-value=0.7020 (Normal)
STA: W=0.9143, p-value=0.4939 (Normal)

=== Paired T-Test & Wilcoxon Test Against RF ===
T-Test STA vs RF: t-stat=0.1192, p-value=0.9109 (Not Significant)
T-Test STA vs ET: t-stat=2.6325, p-value=0.0580 (Not Significant)
T-Test STA vs ADA: t-stat=2.8666, p-value=0.0456 (Significant)
T-Test STA vs GB: t-stat=0.4079, p-value=0.7042 (Not Significant)
T-Test STA vs HGB: t-stat=1.1858, p-value=0.3013 (Not Significant)
T-Test STA vs XGB: t-stat=1.7053, p-value=0.1633 (Not Significant)
T-Test STA vs CB: t-stat=0.7825, p-value=0.4777 (Not Significant)


In [1]:
import numpy as np
from scipy.stats import friedmanchisquare

# Data hasil K-Fold Cross Validation
model_RF = np.array([0.8824, 0.8240, 0.8416, 0.8065, 0.9112])
model_ET = np.array([0.8313, 0.7456, 0.8503, 0.7752, 0.8712])
model_ADA = np.array([0.8008, 0.7785, 0.7538, 0.8257, 0.7993])
model_GB = np.array([0.8910, 0.8257, 0.8120, 0.8432, 0.8745])
model_HGB = np.array([0.8824, 0.8185, 0.7961, 0.8344, 0.8396])
model_XGB = np.array([0.8226, 0.7922, 0.7944, 0.8328, 0.9008])
model_CB = np.array([0.8844, 0.7977, 0.8344, 0.8257, 0.9112])
model_STA = np.array([0.9017, 0.7873, 0.8344, 0.8381, 0.9112])

# Gabungkan data dalam bentuk array (baris = fold, kolom = model)
data = np.array([model_RF, model_ET, model_ADA, model_GB, model_HGB, model_XGB, model_CB, model_STA]).T

# Friedman Test
stat, p_value = friedmanchisquare(*data)

print(f"Friedman Test Statistic: {stat}, p-value: {p_value}")


Friedman Test Statistic: 20.100000000000023, p-value: 0.00047720252711676634


Nemenyi Test

In [39]:
pip install scikit-posthocs

Note: you may need to restart the kernel to use updated packages.Collecting scikit-posthocs



In [6]:
import numpy as np
import scikit_posthocs as sp
import pandas as pd
from scipy.stats import rankdata

# Data hasil K-Fold Cross Validation
model_RF = np.array([0.8824, 0.8240, 0.8416, 0.8065, 0.9112])
model_ET = np.array([0.8313, 0.7456, 0.8503, 0.7752, 0.8712])
model_ADA = np.array([0.8008, 0.7785, 0.7538, 0.8257, 0.7993])
model_GB = np.array([0.8910, 0.8257, 0.8120, 0.8432, 0.8745])
model_HGB = np.array([0.8824, 0.8185, 0.7961, 0.8344, 0.8396])
model_XGB = np.array([0.8226, 0.7922, 0.7944, 0.8328, 0.9008])
model_CB = np.array([0.8844, 0.7977, 0.8344, 0.8257, 0.9112])
model_STA = np.array([0.9017, 0.7873, 0.8344, 0.8381, 0.9112])

# Gabungkan data dalam bentuk array (baris = fold, kolom = model)
data = np.array([model_RF, model_ET, model_ADA, model_GB, model_HGB, model_XGB, model_CB, model_STA]).T

# Buat dataframe dengan nama model
model_names = ["RF", "ET", "ADA", "GB", "HGB", "XGB", "CB", "STA"]
df = pd.DataFrame(data, columns=model_names)

# Nemenyi post-hoc test
nemenyi_results = sp.posthoc_nemenyi_friedman(df.values)

# Tambahkan label model ke hasil
nemenyi_results.columns = model_names
nemenyi_results.index = model_names

print("=== Nemenyi Post-hoc Test Results ===")
print(nemenyi_results)

=== Nemenyi Post-hoc Test Results ===
           RF        ET       ADA        GB       HGB       XGB        CB  \
RF   1.000000  0.815971  0.216050  0.999829  0.994392  0.924296  1.000000   
ET   0.815971  1.000000  0.978833  0.525520  0.996738  0.999996  0.848284   
ADA  0.216050  0.978833  1.000000  0.071599  0.701493  0.924296  0.246911   
GB   0.999829  0.525520  0.071599  1.000000  0.924296  0.701493  0.999584   
HGB  0.994392  0.996738  0.701493  0.924296  1.000000  0.999829  0.996738   
XGB  0.924296  0.999996  0.924296  0.701493  0.999829  1.000000  0.942583   
CB   1.000000  0.848284  0.246911  0.999584  0.996738  0.942583  1.000000   
STA  0.999940  0.570360  0.085327  1.000000  0.942583  0.742183  0.999829   

          STA  
RF   0.999940  
ET   0.570360  
ADA  0.085327  
GB   1.000000  
HGB  0.942583  
XGB  0.742183  
CB   0.999829  
STA  1.000000  
